In [ ]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
# google-colab 1.0.0 requires ipykernel~=5.3.4.
# google-colab 1.0.0 requires ipython~=7.9.0.
# google-colab 1.0.0 requires tornado~=6.0.4.
!pip install --quiet ipykernel==5.3.4 ipython==7.9.0 tornado==6.0.4 oct2py

# install packages
!pip install matpower matpowercaseframes --quiet

In [ ]:
import oct2py
import matpower

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.5.1
matpower version: 7.1.0.2.1.4


In [ ]:
from matpower import start_instance

In [ ]:
m = start_instance()  # runpf require oct2py / matlab

## `runpf` default to `case9.m`

In [ ]:
_ = m.runpf()


MATPOWER Version 7.1, 08-Oct-2020 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.32 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformer

## `runpf` from `mpc` loaded using `loadcase`

In [ ]:
mpc = m.loadcase('case9', verbose=False)
mpc = m.runpf(mpc)

# NOTE:
#   verbose is oct2py arguments to avoid printing from octave


MATPOWER Version 7.1, 08-Oct-2020 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.09 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformer

In [ ]:
mpc.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost', 'order', 'et', 'success', 'iterations'])

In [ ]:
mpc['gencost']

array([[2.000e+00, 1.500e+03, 0.000e+00, 3.000e+00, 1.100e-01, 5.000e+00,
        1.500e+02],
       [2.000e+00, 2.000e+03, 0.000e+00, 3.000e+00, 8.500e-02, 1.200e+00,
        6.000e+02],
       [2.000e+00, 3.000e+03, 0.000e+00, 3.000e+00, 1.225e-01, 1.000e+00,
        3.350e+02]])

## `runopf` require tricks to avoid `<object opf_model>` 

In [ ]:
mpc = m.loadcase('case9', verbose=False)
result = m.runopf(mpc, nout='max_nout')


MATPOWER Version 7.1, 08-Oct-2020 -- AC Optimal Power Flow
  AC OPF formulation: polar voltages, power balance eqns
MATPOWER Interior Point Solver -- MIPS, Version 1.4, 08-Oct-2020
 (using built-in linear solver)
Converged!

Converged in 4.85 seconds
Objective Function Value = 5296.69 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    318.3              -9.6
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt

In [ ]:
f"`result` contains {len(result)} data"

'`result` contains 8 data'

Using `m.runopf(..., nout='max_nout')` will trigger matpower to use:

```octave
[baseMVA, bus, gen, gencost, branch, f, success, et] = runopf(...);
```

You can see matpower `runopf()` docs using:

```ipython
m.runopf?
```

## `runopf` wrapper

In [ ]:
def runopf(mpc, m=None, verbose=False, inplace=True):
    if m is None:
        m = start_instance()
        SHUTDOWN = True
    else:
        SHUTDOWN = False

    # push value to octave client
    m.push('_mpc', mpc)

    # use octave native to run some commands
    # TODO: support mpoption
    m.eval("_mpopt = mpoption('verbose', 2);", verbose=verbose)
    m.eval("_r1 = runopf(_mpc, _mpopt);", verbose=verbose)

    # fech data to python (.eval is used because .pull is not working in acessing field)
    if not inplace:
        mpc = {}

    _extract_result(mpc, m, verbose=verbose)

    if SHUTDOWN:
        m.exit()

    return mpc

def _extract_result(mpc, m, verbose=False):
    mpc['baseMVA'] = m.eval('_r1.baseMVA;', verbose=verbose)
    mpc['version'] = m.eval('_r1.version;', verbose=verbose)
    mpc['bus'] = m.eval('_r1.bus;', verbose=verbose)
    mpc['gen'] = m.eval('_r1.gen;', verbose=verbose)
    mpc['branch'] = m.eval('_r1.branch;', verbose=verbose)
    mpc['gencost'] = m.eval('_r1.gencost;', verbose=verbose)

In [ ]:
mpc = m.loadcase('case9', verbose=False)
mpc = runopf(mpc, m=m, verbose=True, inplace=True)  # using runopf wrapper


MATPOWER Version 7.1, 08-Oct-2020 -- AC Optimal Power Flow
  AC OPF formulation: polar voltages, power balance eqns
MATPOWER Interior Point Solver -- MIPS, Version 1.4, 08-Oct-2020
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0      8363.125                   0.155       0.1765      57.1483            0
  1     5307.5519    0.68632   0.00900309      8.30505      15.6381     0.166397
  2     5391.0949    0.24288  0.000359195     0.630698      2.20652   0.00545763
  3     5352.5722   0.088637   3.7341e-05     0.141404     0.216699   0.00250292
  4     5315.4636    0.17189  0.000106315    0.0190754      0.02128    0.0024171
  5     5312.3257   0.066353  0.000123278   0.00135236   0.00351135  0.000204886
  6     5301.3809    0.19111   0.00120494   0.00174642  0.000962847  0.000714764
  7     5296.7947    0.11693  0.000322236  7.55989e-05  0.0

In [ ]:
mpc.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost'])

In [ ]:
mpc['gencost']

array([[2.000e+00, 1.500e+03, 0.000e+00, 3.000e+00, 1.100e-01, 5.000e+00,
        1.500e+02],
       [2.000e+00, 2.000e+03, 0.000e+00, 3.000e+00, 8.500e-02, 1.200e+00,
        6.000e+02],
       [2.000e+00, 3.000e+03, 0.000e+00, 3.000e+00, 1.225e-01, 1.000e+00,
        3.350e+02]])